In [2]:
import requests
from xml.etree import ElementTree
from datetime import datetime, date
from pathlib import Path
import xarray as xr
import numpy as np
import os


In [4]:
# Constants
BASE_CATALOG_URL = "https://esgf-data.dwd.de/thredds/catalog/esgf3/data/climatepredictionsde/seasonal/output/public/DE-0075x005/DWD/GCFS21/svhYYYY{month:02}01/sfc{year}{month:02}01/{scenario}/DWD-EPISODES2022/v1-r1/day/{variable}/"
SCENARIOS = ['r1i1p1', 'r2i1p1', 'r3i1p1']
# SCENARIOS = ['r1i1p1']
VARIABLES = ['hurs', 'pr', 'psl', 'rsds', 'sfcWind', 'tas', 'tasmax', 'tasmin']
THREDDS_NAMESPACE = {"thredds": "http://www.unidata.ucar.edu/namespaces/thredds/InvCatalog/v1.0"}


In [7]:
local_path = '/home/colja/01_Code/01_SWN/Spreewasser-N/app/monica/climate_netcdf_forecast/'


In [8]:
for scenario in SCENARIOS:
    folder_path = f"{local_path}/{scenario}/"
    netcdf_paths = [f'{folder_path}/{nc}' for nc in os.listdir(folder_path) if nc.endswith('.nc')]
    ds = xr.open_mfdataset(netcdf_paths, combine='by_coords', compat='override')
    dates = netcdf_paths[0].split('_')[-1].split('.')[0]
    filename = f'forecast_{scenario}_{dates}.nc'
    file_path = f"{local_path}/{filename}"
    ds.to_netcdf(file_path)
    ds.close()

In [10]:
folder_path = f"./r1i1p1"
netcdf_paths = [f'{folder_path}/{nc}' for nc in os.listdir(folder_path) if nc.endswith('.nc')]

In [16]:
netcdf_paths.pop()

'./r1i1p1/hurs_day_GCFS21--DWD-EPISODES2022--DE-0075x005_sfc20250201_r1i1p1_20250201-20250731.nc'

In [17]:
ds = xr.open_mfdataset(netcdf_paths, combine='by_coords', compat='override')

In [18]:
netcdf_paths

['./r1i1p1/pr_day_GCFS21--DWD-EPISODES2022--DE-0075x005_sfc20250201_r1i1p1_20250201-20250731.nc',
 './r1i1p1/tasmax_day_GCFS21--DWD-EPISODES2022--DE-0075x005_sfc20250201_r1i1p1_20250201-20250731.nc',
 './r1i1p1/sfcWind_day_GCFS21--DWD-EPISODES2022--DE-0075x005_sfc20250201_r1i1p1_20250201-20250731.nc',
 './r1i1p1/tasmin_day_GCFS21--DWD-EPISODES2022--DE-0075x005_sfc20250201_r1i1p1_20250201-20250731.nc',
 './r1i1p1/rsds_day_GCFS21--DWD-EPISODES2022--DE-0075x005_sfc20250201_r1i1p1_20250201-20250731.nc']